# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Machine learning final project

**Date**: November 23rd 2025

**Student Name**: Jaime Antonio Contreras Barragan

**Professor**: Pablo Camarillo Ramirez

# Create SparkSession

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Final Project ML") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")
# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 04:10:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# ML algorimo a usar


Primero que nada quiero poder adivinar si una reseña va a ser positiva, negativa o neutral, sin tener que leer la reseña del celular como tal. Para empezar tenemos 3 clasificaciones en el propio dataset, por lo tanto ya sabemos que tipo de algoritmo tenemos que usar, en este caso el de clasificacion.
### SVM
Ahora, porque SVM? Para empezar ya contamos como tal con las clases, y son 3, ademas que como tal quiero que clasifique, no que me de una probabilidad.

# Descripcion de dataset:


Kaggle link: https://www.kaggle.com/datasets/mohankrishnathalla/mobile-reviews-sentiment-and-specification

Es el mismo dataset de las practicas finales anteriores.
Este es un dataset de reviews de celulares 2024-2025. 
En este se tiene en cuenta la review como tal, calificacion general, y de caracteristicas del telefono, datos de reseñador, (y lo que nos importa para esta practica) datos relacionados con la reseña, longitud, cantidad de palabras, votos.

In [2]:
ls -l /opt/spark/work-dir/data/Mobile_Reviews

total 9824
-rwxrwxrwx 1 root root 10056684 Oct 23 03:20 Mobile_Reviews_Sentiment.csv*
drwxr-xr-x 1 root root     4096 Nov 16 23:30 _spark_metadata/


In [3]:
from pyspark.sql.functions import col,desc,count, rand
from jaimodule.spark_utils import SparkUtils

schema = SparkUtils.generate_schema([
    ("review_id","int"),
    ("customer_name","string"),
    ("age", "int"),
    ("brand", "string"),
    ("model", "string"),
    ("price_usd", "float"),
    ("price_local", "string"),
    ("currency", "string"),
    ("exchange_rate_to_usd", "float"),
    ("rating", "int"),
    ("review_text", "string"),
    ("sentiment", "string"),
    ("country", "string"),
    ("language", "string"),
    ("review_date", "date"),
    ("verified_purchase", "boolean"),
    ("battery_life_rating", "int"),
    ("camera_rating", "int"),
    ("performance_rating", "int"),
    ("design_rating", "int"),
    ("display_rating", "int"),
    ("review_length", "int"),
    ("word_count", "int"),
    ("helpful_votes", "int"),
    ("source", "string")
])

# Import your module
#tuve que poner el path completo por el metadata spark
base_path = "/opt/spark/work-dir/data/Mobile_Reviews/Mobile_Reviews_Sentiment.csv"
mobile_reviews_df = spark.read \
                .option("header", "true") \
                .schema(schema) \
                .csv(base_path) 

data_df= mobile_reviews_df.select(
    col("sentiment"),
    col("word_count"),
    col("review_length"),
    col("helpful_votes")
    #col("price_usd")
)

data_df.groupBy("sentiment").count().show()
print(data_df.count())

+---------+-----+
|sentiment|count|
+---------+-----+
| Positive|27540|
|  Neutral|12549|
| Negative| 9911|
+---------+-----+

50000


Como vemos el dataset no esta balanceado, se podria decir que esta entre 1/2, 1/4 y 1/5 mas o menos.

# ML Training Process

## SVM

## Collect Data

In [4]:
positive_df = data_df.filter(col("sentiment") == "Positive").limit(9911)
neutral_df  = data_df.filter(col("sentiment") == "Neutral").limit(9911)
negative_df = data_df.filter(col("sentiment") == "Negative").limit(9911)

balanced_df = positive_df.union(negative_df).union(neutral_df)
ordered_df = balanced_df.orderBy(rand())

#Aqui solo balanceo el df, basandome en el grupo mas pequeño para que sean del mismo tamaño
print(ordered_df.count())
ordered_df.show(10)

29733


[Stage 20:=============================>                            (1 + 1) / 2]

+---------+----------+-------------+-------------+
|sentiment|word_count|review_length|helpful_votes|
+---------+----------+-------------+-------------+
| Positive|        13|           69|            5|
|  Neutral|        14|           71|            1|
| Negative|         7|           45|            2|
|  Neutral|         9|           60|            2|
|  Neutral|        13|           82|            2|
| Negative|        10|           46|            2|
| Negative|         9|           56|            2|
| Negative|         9|           57|            1|
| Positive|        10|           55|            3|
|  Neutral|        11|           62|            1|
+---------+----------+-------------+-------------+
only showing top 10 rows


# Assemble the features into a single vector column

In [5]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

indexer = StringIndexer(inputCol="sentiment", outputCol="label")
indexer_model = indexer.fit(ordered_df)
indexed_df = indexer_model.transform(ordered_df)

#assembler = VectorAssembler(inputCols = ["word_count","review_length","helpful_votes","price_usd"],outputCol = "features")
#Se elegieron las siguierntes columnas por lo siguiente:
#-Descartamos ratings, ya que en la relacion entre buen rating y mal rating es obvio, para que no sea el unico dato importante
#-utilizamos todos los datos relacionados al review, sin tocar el review como tal, ya que tendriamos que tokenizarlo
#-Word count: Cantidad de palabras
#-Review length: Longitud de la review
#-helpful_votes: Si a sido relevante para otros usuarios
#Utilizamos estos parametros relacionados al review para sin necesitar el review como tal poder saber en que categoria caera.
assembler = VectorAssembler(inputCols = ["word_count","review_length","helpful_votes"],outputCol = "features")

data_final = assembler.transform(indexed_df).select("label", "features")

data_final.show(5, truncate=False)

[Stage 69:>                                                         (0 + 1) / 1]

+-----+---------------+
|label|features       |
+-----+---------------+
|2.0  |[13.0,69.0,5.0]|
|1.0  |[14.0,71.0,1.0]|
|0.0  |[7.0,45.0,2.0] |
|1.0  |[9.0,60.0,2.0] |
|1.0  |[13.0,82.0,2.0]|
+-----+---------------+
only showing top 5 rows


## Data splitting

In [6]:
#PARA TEST Y PARA PRUEBA
train_df, test_df = data_final.randomSplit([0.8,0.2], seed = 57)
train_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 7886|
|  1.0| 7901|
|  2.0| 7879|
+-----+-----+



## Create ML Model

In [7]:
from pyspark.ml.classification import OneVsRest, LinearSVC

lsvc = LinearSVC(maxIter=50, regParam = 0.07) #0.73
#lsvc = LinearSVC(maxIter=25, regParam = 0.1) #0.724
#lsvc = LinearSVC(maxIter=50, regParam = 0.1) #0.728
#lsvc = LinearSVC(maxIter=50, regParam = 0.05) 0.727
#lsvc = LinearSVC(maxIter=50, regParam = 0.01) 0.724
#lsvc = LinearSVC(maxIter=25, regParam = 0.05) 
#lsvc = LinearSVC(maxIter=100, regParam = 0.01) 0.75
#lsvc = LinearSVC(maxIter=10, regParam = 0.01) 0.723
ovr = OneVsRest(classifier=lsvc)


## Train ML Model

In [8]:
ovr_model = ovr.fit(train_df)

predictions = ovr_model.transform(test_df)

25/11/24 04:10:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


## Persist the model

In [9]:
ova_model_path = "/opt/spark/work-dir/data/mlmodels/svm/svm_ova_fp"
ovr_model.write().overwrite().save(ova_model_path)

# ML EVALUATION

## Predictions

In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol = "label", predictionCol = "prediction")

accuracy = evaluator.evaluate(predictions,
                             {evaluator.metricName: "accuracy"})
#Como esta balanceado usamos este
print(f"Accuracy: {accuracy}")

#no balanceados
f1 = evaluator.evaluate(predictions,
                       {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}")

Accuracy: 0.7285313993736607


F1 Score: 0.7280765961149458


In [11]:
sc.stop

<bound method SparkContext.stop of <SparkContext master=spark://spark-master:7077 appName=Final Project ML>>